In [2]:
import sqlite3

# Connect to (or create) the database
conn = sqlite3.connect("employees.db")

# Open and read the SQL script
with open("employee_data.sql", "r") as file:
    sql_script = file.read()

# Execute the entire SQL script
cursor = conn.cursor()
cursor.executescript(sql_script)

# Commit changes and close the connection
conn.commit()
conn.close()

print("SQL script executed successfully.")


SQL script executed successfully.


In [4]:
conn = sqlite3.connect("employees.db")
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)

# Preview data (replace `your_table_name` with the actual name)
cursor.execute("SELECT * FROM employees LIMIT 5;")
rows = cursor.fetchall()
print("Sample data:", rows)

conn.close()


Tables: [('employees',)]
Sample data: [(1, 'John', 'Engineering', 75000), (2, 'Alice', 'Marketing', 62000), (3, 'Bob', 'Engineering', 68000), (4, 'Diana', 'HR', 60000), (5, 'Eve', 'Marketing', 64000)]


In [2]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
from langchain.llms.fake import FakeListLLM
import sqlite3

# Connect to SQLite database
def query_employee_db(sql_query: str) -> str:
    try:
        conn = sqlite3.connect("employees.db")
        cursor = conn.cursor()
        cursor.execute(sql_query)
        results = cursor.fetchall()
        conn.close()
        return str(results)
    except Exception as e:
        return f"Database error: {e}"

# Define the database tool
@tool
def employee_db_tool(query: str) -> str:
    """
    Useful for answering questions about employees from the company database.
    Input should be a full SQL query string (e.g., 'SELECT * FROM employees WHERE department = "Marketing"')
    """
    return query_employee_db(query)

# Fake responses that simulate the LLM output
# In real use, LLM would generate these dynamically
responses = [
    "Action: employee_db_tool\nAction Input: \"SELECT salary FROM employees WHERE name = 'Diana'\"",
    "Observation: [(75000,)]\nThought: I now know the final answer\nFinal Answer: John's salary is 75000."
]

llm = FakeListLLM(responses=responses)

# Initialize agent
tools = [employee_db_tool]

agent = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run query
question = "What is the salary of John?"
response = agent.run(question)

print("\nResponse:", response)




> Entering new AgentExecutor chain...
Action: employee_db_tool
Action Input: "SELECT salary FROM employees WHERE name = 'Diana'"
Observation: [(60000,)]
Thought:Observation: [(75000,)]
Thought: I now know the final answer
Final Answer: John's salary is 75000.

> Finished chain.

Response: John's salary is 75000.
